# Sistem Penjadwalan Praktikum Otomatis menggunakan Algoritma Genetika dan Tabu Search.

# Setup

In [38]:
import os
import sys
import django
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "LabTimetablingAPI.settings")
django.setup()
from django.conf import settings
#set debug to true
settings.DEBUG = True

#logging
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## Data

Import data dari model yang telah dibuat sebelumnya.

```python

In [39]:
from scheduling_data.models import Laboratory, Module, Participant, Group, Assistant, Chapter

Ekstrak data dari model yang telah dibuat sebelumnya.

```python

In [40]:
class LaboratoryData:

    def __init__(self, lab = Laboratory):
        self._lab = lab
        self._laboratories = lab.objects.all()

    @property
    def laboratories(self): 
        return self._laboratories.values()
    
    def get_assistant(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        assistants = laboratory.assistants.all()
        return assistants.values()
    
    def get_module(self,id):
        laboratory = self._laboratories.filter(id=id).first()
        modules = laboratory.modules.all()
        return modules.values()
    
class ModuleData:

    def __init__(self, module = Module):
        self._module = module
        self._modules = module.objects.all()

    @property
    def modules(self): 
        return self._modules.values()
    
    def get_module(self,id):
        module = self._modules.filter(id=id).first()
        return module
    
    def get_module_chapter(self, id):
        module = self.get_module(id)
        return module.chapters.all().values()
    
    def get_group(self, id):
        module = self.get_module(id)
        return module.groups.all().values()
    
    def get_laboratory(self, id):
        module = self.get_module(id)
        return module.laboratory
    
        
class ChapterData:

    def __init__(self, chapter = Chapter):
        self._chapter = chapter
        self._chapters = chapter.objects.all()

    @property
    def chapters(self): 
        return self._chapters.values()
    
    def get_chapter(self,id):
        chapter = self._chapters.filter(id=id).first()
        return chapter
    
    def get_chapter_module(self, id):
        chapter = self.get_chapter(id)
        return chapter.module

class ParticipantData:

    def __init__(self, participant = Participant):
        self._participant = participant
        self._participants = participant.objects.all()
    
    @property
    def participants(self):
        return self._participants.values()
    
    def group(self, id):
        group_memberships = self._participants.filter(id=id).first().group_memberships.all()
        groups = []
        for group_membership in group_memberships:
            group = group_membership.group
            groups.append(group)
        return groups
    
class GroupData:
    def __init__(self, group = Group):
        self._group = group
        self._groups = group.objects.all()

    @property
    def groups(self):
        return self._groups.values()
    
    def get_group(self, id):
        group = self._groups.filter(id=id).first()
        return group
    
    def get_module_group(self, module_id):
        groups = self._groups.filter(module_id=module_id)
        return groups.values()
    
    def participants(self, id):
        group = self._groups.filter(id=id).first()
        group_memberships = group.group_memberships.all()
        participants = []
        for group_membership in group_memberships:
            participant = group_membership.participant
            participants.append(participant)
        return participants
    
    def get_participant_schedule(self, id):
        participants = self.participants(id)
        participant_schedule = []
        for participant in participants:
            participant_schedule.append(participant.regular_schedule)
        return participant_schedule
    
    def get_group_schedule(self, id):
        participant_schedule = self.get_participant_schedule(id)
        if len(participant_schedule) == 0:
            return {}
        days = participant_schedule[0].keys()
        merged_schedule = {day: {} for day in days}
        for day in days:
            for time_slot in participant_schedule[0][day]:
                is_available = all(schedule[day][time_slot] for schedule in participant_schedule)
                merged_schedule[day][time_slot] = is_available
        return merged_schedule
                
            
class AssistantData:

    def __init__(self, assistant = Assistant):
        self._assistant = assistant
        self._assistants = assistant.objects.all()

    @property
    def assistants(self):
        return self._assistants.values()

    
    def get_assistant(self, id):
        assistant = self._assistants.filter(id=id).first()
        return assistant
    
    def get_assistant_modules(self,id):
        assistant = self._assistant.filter(id=id).first()
        assistant_memberships = assistant.assistant_memberships.all()
        modules = []
        for assistant_membership in assistant_memberships:
            module = assistant_membership.module
            modules.append(module)
        return modules

Constants

```python

In [41]:
# Constant.py
class Constant:
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
    shifts = ["Shift1", "Shift2", "Shift3", "Shift4", "Shift5", "Shift6"]

# Algoritma Genetika

## Gene Representation

```python

In [42]:
from collections import namedtuple
TimeSlot = namedtuple("TimeSlot", ["date", "day", "shift"])

class Gene:
    def __init__(self, laboratory, module, module_chapter, group, assistant, time_slot: TimeSlot):
        self.laboratory = laboratory
        self.module = module
        self.module_chapter = module_chapter
        self.group = group
        self.assistant = assistant
        self.time_slot = time_slot
        
    def __repr__(self):
        return f"Gene(lab={self.laboratory}, module={self.module}, module_chapter={self.module_chapter}, group={self.group}, assistant={self.assistant}, time_slot={self.time_slot})"

Test Gene Representation

```python

In [43]:
# Test Gene

# Assigning data to variables
lab_data = LaboratoryData()
module_data = ModuleData()
participant_data = ParticipantData()
group_data = GroupData()
assistant_data = AssistantData()
chapter_data = ChapterData()

gene = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[0]["id"],assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift1"))
gene2 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[1]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Friday", "Shift2"))
gene3 = Gene(lab_data.laboratories[0]["id"], module_data.modules[0]["id"], chapter_data.chapters[0]["id"], group_data.groups[2]["id"], assistant_data.assistants[0]["id"], TimeSlot("2021-01-01", "Monday", "Shift1"))

### Gene Level Constraints

```python

In [44]:
# Constraint Class for Constraint Function
# 1. Ensure that each chapter is assigned to the correct module.
# 2. Ensure that each module is assigned to the correct lab.
# 3. Ensure that each group is assigned to the correct module.
# 4. Ensure that each assistant is assigned to the correct lab.

# import LaboratoryData, ModuleData, GroupData, AssistantData, ChapterData

class ConstraintChecker:
    def __init__(self, gene : Gene, **kwargs):
        kwargs.setdefault("lab_data", LaboratoryData())
        kwargs.setdefault("module_data", ModuleData())
        kwargs.setdefault("group_data", GroupData())
        kwargs.setdefault("assistant_data", AssistantData())
        kwargs.setdefault("chapter_data", ChapterData())
        self.gene = gene
        self.lab_data = lab_data
        self.module_data = module_data
        self.group_data = group_data
        self.assistant_data = assistant_data
        self.chapter_data = chapter_data

    def chapter_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        chapter = self.chapter_data.get_chapter(self.gene.module_chapter)
        return module == chapter.module
    
    def module_lab_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        return module.laboratory.id == self.gene.laboratory
    
    def group_module_constraint(self):
        module = self.module_data.get_module(self.gene.module)
        group = self.group_data.get_group(self.gene.group)
        return module == group.module
    
    def assistant_lab_constraint(self):
        assistant = self.assistant_data.get_assistant(self.gene.assistant)
        return assistant.laboratory.id == self.gene.laboratory
    
    def schedule_constraint(self):
        return group_data.get_group_schedule(self.gene.group)[self.gene.time_slot.day][self.gene.time_slot.shift]
    
    def check(self):
        return self.chapter_module_constraint() and self.module_lab_constraint() and self.group_module_constraint() and self.assistant_lab_constraint()

Test Gene Level Constraints

```python

In [45]:
# Test ConstraintChecker
ConstraintChecker(gene).check()

True

## Chromosome Representation

```python

In [46]:
# Chromosome.py
# Chromosome Representation
# 1. Gene
# 2. Fitness

class Chromosome:
    def __init__(self, genes):
        self.genes = genes
        self.fitness = None

    def __repr__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness})"
    
    def __str__(self):
        return f"Chromosome(genes={self.genes}, fitness={self.fitness})"
    

### Chromosome Level Constraints

```python

Objective class
1. Minimize conflict: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.
2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.
3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.

#### Objective Class for Fitness Function
1. Minimize Conflicts: Minimize conflicts in the schedule by ensuring that no two groups are assigned to the same lab and time slot simultaneously.

In [47]:
#minimize_conflicts.py
from collections import namedtuple

class MinimizeConflicts:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.conflicts = namedtuple("Conflicts", ["time_slot", "lab"])
        self.conflicts_time_slot = set()
        self.assigned_time_slot = set()
        self.conflict_count = 0


    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Conflicts Time Slot: {self.conflicts_time_slot}, Assigned Time Slot: {self.assigned_time_slot}, Conflict Count: {self.conflict_count}"
    
    def __eq__(self, other):
        return self.chromosome == other.chromosome and self.conflicts_time_slot == other.conflicts_time_slot and self.assigned_time_slot == other.assigned_time_slot and self.conflict_count == other.conflict_count
    
    def calculate_conflicts(self):
        for gene in self.chromosome.genes:
            if (gene.laboratory, gene.time_slot) in self.assigned_time_slot:
                self.conflicts_time_slot.add(self.conflicts(gene.time_slot, gene.laboratory))
                self.conflict_count += 1
            else:
                self.assigned_time_slot.add((gene.laboratory, gene.time_slot))
        return self.conflict_count

    def calculate_fitness(self):
        if settings.DEBUG:
            logger.info(f"Calculating conflicted group time slots for chromosome {self.chromosome}")
        self.calculate_conflicts()
        return self.conflict_count

2. Maximize Resource Utilization: Maximize the utilization of assistants by distributing tasks evenly among them. Each assistant should be assigned to a balanced number of groups and shift to avoid overloading.


In [48]:
from collections import defaultdict

class MaximizeResourceUtilization:
    def __init__(self, chromosome:list, max_groups_per_assistant, max_shift_per_assistant):
        self.chromosome = chromosome
        self.max_groups_per_assistant = max_groups_per_assistant
        self.max_shift_per_assistant = max_shift_per_assistant
        self.assistants_assignments = defaultdict(list)
        self.assistants_shifts = defaultdict(list)
        self.overload_penalty = 0

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Max Groups Per Assistant: {self.max_groups_per_assistant}, Max Shift Per Assistant: {self.max_shift_per_assistant}, Assistants Assignments: {self.assistants_assignments}, Assistants Shifts: {self.assistants_shifts}, Overload Penalty: {self.overload_penalty}"
    
    def calculate_assistants_assignments(self):
        self.assistants_assignments.clear()
        for gene in self.chromosome.genes:
            self.assistants_assignments[gene.assistant].append(gene.group)
        return self.assistants_assignments
    
    def calculate_assistants_shifts(self):
        self.assistants_shifts.clear()
        for gene in self.chromosome.genes:
            self.assistants_shifts[gene.assistant].append(gene.time_slot)
        return self.assistants_shifts
    
    def calculate_overload_penalty(self):
        self.overload_penalty = 0
        self.calculate_assistants_assignments()
        self.calculate_assistants_shifts()
        for assistant , assignments in self.assistants_assignments.items():
            assignments_penalty = max(0, len(assignments) - self.max_groups_per_assistant)
            shift_penalty = (max(0, len(self.assistants_shifts[assistant]) - self.max_shift_per_assistant))
            self.overload_penalty += assignments_penalty + shift_penalty
        return self.overload_penalty
    
    def calculate_fitness(self):
        if settings.DEBUG:
            logger.info(f"Calculating overload penalty for assistant distribution: {self.chromosome}")
        self.calculate_overload_penalty()
        return self.overload_penalty

3. Maximize Participant Satisfaction: Maximize the satisfaction of participants by ensuring that each participant is assigned to a time slot that is available to them.


In [49]:
from collections import defaultdict

class ParticipantAvailability:
    def __init__(self, chromosome, group_data: GroupData):
        self.chromosome = chromosome
        self.group_data = group_data
        self.unsatified_participants = defaultdict(list)
        self.unsatified_participants_count = 0

    def calculate_unsatisfied_participants(self):
        self.unsatified_participants.clear()
        for gene in self.chromosome.genes:
            participant_availability = group_data.get_group_schedule(gene.group)
            if not participant_availability[gene.time_slot.day][gene.time_slot.shift]:
                self.unsatified_participants[gene.group].append(gene.time_slot)
                self.unsatified_participants_count += 1
        return self.unsatified_participants_count
    
    def calculate_fitness(self):
        if settings.DEBUG:
            logger.info(f"Calculating unsatisfied participants schedule for chromosome {self.chromosome}")
        self.calculate_unsatisfied_participants()
        return self.unsatified_participants_count

# Population Initialization

In [50]:
# import fitness_function

class Population:
    def __init__(self, chromosome, fitness_function):
        self.chromosome = chromosome
        self.fitness_function = fitness_function

    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def __str__(self):
        return f"Chromosome: {self.chromosome}, Fitness Function: {self.fitness_function}"
    
    def calculate_fitness(self):
        for chromosome in self.chromosome:
            chromosome.fitness = self.fitness_function.calculate_fitness(chromosome)


# Helper Functions

## Factory Method

```python

In [51]:
# Factory.py
import numpy as np
from math import ceil
from collections import namedtuple
from datetime import timedelta
# import LaboratoryData, ModuleData, GroupData, AssistantData, ChapterData
# import Constant, ConstraintChecker, Gene, Chromosome, Population

class Factory:
    def __init__(self, **kwargs):
        
        self.lab_data = kwargs.get("lab_data", LaboratoryData())
        self.module_data = kwargs.get("module_data", ModuleData())
        self.group_data = kwargs.get("group_data", GroupData())
        self.assistant_data = kwargs.get("assistant_data", AssistantData())
        self.chapter_data = kwargs.get("chapter_data", ChapterData())

    def create_time_slot(self, module_id):
        start = self.module_data.get_module(module_id).start_date
        end = self.module_data.get_module(module_id).end_date
        duration = (end - start).days + 1
        weeks_duration = ceil(duration / 7)
        week = np.random.randint(0, weeks_duration)
        day = np.random.choice(Constant.days)
        shift = np.random.choice(Constant.shifts)
        date = start + timedelta(days=week*7 + Constant.days.index(day))
        return TimeSlot(date, day, shift)

    def generate_gene(self):
        while True:
            laboratory = np.random.choice(self.lab_data.laboratories)["id"]
            module = np.random.choice(self.lab_data.get_module(laboratory))["id"]
            module_chapter = np.random.choice(self.module_data.get_module_chapter(module))["id"]
            group = np.random.choice(self.module_data.get_group(module))["id"]
            assistant = np.random.choice(self.lab_data.get_assistant(laboratory))["id"]

            time_slot = self.create_time_slot(module_id=module)

            gene = Gene(laboratory, module, module_chapter, group, assistant, time_slot)
            constraint_checker = ConstraintChecker(gene)
            if constraint_checker.check():
                return gene

    def calculate_chromosome_length(self):
        chromosome_size = 0
        for module in self.module_data.modules:
            module_chapter_count = self.module_data.get_module_chapter(module["id"]).count()
            group_count = self.module_data.get_group(module["id"]).count()
            chromosome_size += module_chapter_count * group_count
        return chromosome_size
    
    def generate_chromosome(self):
        if settings.DEBUG:
            logger.info("Generating chromosome...")
        chromosome_length = self.calculate_chromosome_length()
        genes = []
        for _ in range(chromosome_length):
            gene = self.generate_gene()
            genes.append(gene)
        return Chromosome(genes)

    def generate_population(self, population_size):
        if settings.DEBUG:
            logger.info("Generating population...")
        chromosome = []
        for _ in range(population_size):
            chromosome.append(self.generate_chromosome())
        return Population(chromosome, None)

Test Factory

In [52]:
#test Factory
factory = Factory()
factory.generate_population(1)

INFO:__main__:Generating population...
INFO:__main__:Generating chromosome...


Chromosome: [Chromosome(genes=[Gene(lab=1, module=1, module_chapter=4, group=9, assistant=1, time_slot=TimeSlot(date=datetime.datetime(2022, 10, 28, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Friday', shift='Shift1')), Gene(lab=1, module=1, module_chapter=5, group=28, assistant=6, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 1, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Tuesday', shift='Shift1')), Gene(lab=1, module=1, module_chapter=6, group=12, assistant=6, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 10, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Thursday', shift='Shift2')), Gene(lab=1, module=1, module_chapter=3, group=27, assistant=1, time_slot=TimeSlot(date=datetime.datetime(2022, 11, 21, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), day='Monday', shift='Shift3')), Gene(lab=1, module=1, module_chapter=8, group=30, assistant=4, time_slot=TimeSlot(date=datetime.datetime(2022, 9, 12, 7, 30, 0, 530000, tzinfo=datetime.timezone.utc), d

## Fitness Function

```python

In [53]:
class FitnessFunction:
    def __init__(self, all: bool = True):
        self.minimize_conflicts = MinimizeConflicts
        self.maximize_resource_utilization = MaximizeResourceUtilization
        self.participant_availability = ParticipantAvailability
        self.all = all

        # MaximizeResourceUtilization Parameters
        self.max_groups_per_assistant = 2
        self.max_shift_per_assistant = 16
    
    def calculate_all_fitness(self, chromosome: Chromosome):
        fitness = []
        fitness.append(self.minimize_conflicts(chromosome).calculate_fitness())
        fitness.append(self.maximize_resource_utilization(chromosome, self.max_groups_per_assistant, self.max_shift_per_assistant).calculate_fitness())
        fitness.append(self.participant_availability(chromosome, GroupData()).calculate_fitness())
        return sum(fitness)

    
    def calculate_fitness(self, chromosome: Chromosome):
        if settings.DEBUG:
            logger.info(f"Calculating fitness for chromosome {chromosome}")
        fitness = self.calculate_all_fitness(chromosome)
        return fitness

Test Fitness Function

In [54]:
#test FitnessFunction
# factory = Factory()
# population = factory.generate_population(2)
# fitness_function = FitnessFunction
# population.fitness_function = fitness_function

In [55]:
# population.calculate_fitness()

## Operator

```python

### Genetic Operators

```python

#### Mutation

```python

In [56]:
from copy import deepcopy
from random import random, randint, choice

class Mutation:
    def __init__(self, parent: Chromosome, **kwargs):
        self.parent = parent
        self.mutation_rate = kwargs.get("mutation_rate", 0.5)
        self.mutation_size = kwargs.get("mutation_size", 1)
        self.factory = kwargs.get("factory", Factory())


    def mutate(self) -> Chromosome:
        if settings.DEBUG:
            logger.info("Mutating...")
        if random() < self.mutation_rate:
            chromosome = deepcopy(self.parent)
            for _ in range(self.mutation_size):
                mutation_point = self.mutation_point()
                chromosome.genes[mutation_point] = self.factory.generate_gene()
                if settings.DEBUG:
                    logger.info(f"Mutated chromosome: {chromosome}")
            return chromosome
        else:
            if settings.DEBUG:
                logger.info(f"Chromosome not mutated: {self.parent}")
            return self.parent
        
    def mutation_point(self):
        return randint(0, len(self.parent.genes) - 1)

Test Mutation

In [57]:
#test Mutation
# factory = Factory()
# population = factory.generate_population(1)
# population.chromosome[0].genes
# mutation = Mutation(population.chromosome[0].genes)
# a = mutation.mutate()

#### Cross Over

```python

In [58]:
class Crossover:
    def __init__(self, parent1: Chromosome, parent2: Chromosome, **kwargs):
        self.parent1 = parent1
        self.parent2 = parent2

        self.crossover_probability = kwargs.get("crossover_probability", 0.5)
        self.number_of_crossover_points = kwargs.get("number_of_crossover_points", 1)
        

    def crossover(self) -> Chromosome:
        if settings.DEBUG:
            logger.info("Crossover...")

        if random() > self.crossover_probability:
            if settings.DEBUG:
                logger.info("No crossover")
            return self.parent1, self.parent2

        chromosome1 = deepcopy(self.parent1)
        chromosome2 = deepcopy(self.parent2)

        for _ in range(self.number_of_crossover_points):
            point = self.crossover_point()
            if settings.DEBUG:
                logger.info(f"Crossover point: {point}")
            chromosome1.genes[point], chromosome2.genes[point] = chromosome2.genes[point], chromosome1.genes[point] 
            if settings.DEBUG:
                logger.info(f"Chromosome 1: {chromosome1}")
                logger.info(f"Chromosome 2: {chromosome2}")


        return chromosome1, chromosome2
    
    def crossover_point(self):
        return randint(0, len(self.parent1.genes) - 1)
        

Test Cross Over

In [59]:
#test Crossover
# factory = Factory()
# population = factory.generate_population(2)
# population.chromosome[0].genes
# population.chromosome[1].genes
# crossover = Crossover(population.chromosome[0].genes, population.chromosome[1].genes)
# a, b = crossover.crossover()
# a, b

#### Repair

```python

In [60]:
class Repair:
    def __init__(self, parent: Chromosome, **kwargs):
        self.parent = parent
        self.factory = kwargs.get("factory", Factory())
        
    def repair(self) -> Chromosome:
        logger.info(type(self.parent))
        if settings.DEBUG:
            logger.info("Repairing...")
        chromosome = deepcopy(self.parent)
        logger.info(type(chromosome))
        for gene in chromosome.genes:
            constraint_checker = ConstraintChecker(gene)
            if not constraint_checker.check():
                if settings.DEBUG:
                    logger.info(f"Gene {gene} is not feasible, repairing...")
                feasible_time_slot = self.factory.create_time_slot(gene.module)
                gene.time_slot = feasible_time_slot
                if settings.DEBUG:
                    logger.info(f"Feasible time slot: {gene.time_slot}")
        return chromosome
    
    def find_feasible_solution(self, gene, max_iteration=100):
        # if settings.DEBUG:
        #     logger.info("Finding feasible solution...")

        new_gene = deepcopy(gene)
        for _ in range(max_iteration):
            feasible_time_slot = self.factory.create_time_slot(gene.module)
            new_gene.time_slot = feasible_time_slot
            constraint_checker = ConstraintChecker(new_gene)
            if constraint_checker.schedule_constraint():
                # if settings.DEBUG:
                #     logger.info("Found!")
                return feasible_time_slot
            
        if settings.DEBUG:
            logger.info("No feasible solution found, returning None...")
        return None

Test Repair

In [61]:
#test Repair
# factory = Factory()
# population = factory.generate_population(1)
# population.chromosome[0].genes
# repair = Repair(population.chromosome[0])
# a = repair.repair()

#### Selection

```python

In [62]:
import numpy as np
from copy import deepcopy
from collections import defaultdict

class Selection:
    '''
    Description:
    Selection class for selecting chromosomes from population
    Selection methods:
    1. Tournament selection
    2. Elitism selection
    3. Roulette selection
    '''
    def __init__(self, population: Population, **kwargs):
        self.population = population
        self.tournament_size = kwargs.get("tournament_size", 3)
        self.elitism_size = kwargs.get("elitism_size", 1)
        self.elitism_offset = kwargs.get("elitism_offset", 0)
        self.elitism_range = kwargs.get("elitism_range", 1)
        self.elitism_probability = kwargs.get("elitism_probability", 0.5)
        self.method = kwargs.get("method", "tournament")
        
    def select(self):
        if settings.DEBUG:
            logger.info("Selecting...")
        if self.method == "tournament":
            return self.tournament_selection()
        elif self.method == "elitism":
            return self.elitism_selection()
        elif self.method == "roulette":
            return self.roulette_selection()
        else:
            raise ValueError("Invalid selection method")
    
    def elitism_selection(self) -> Chromosome:
        if settings.DEBUG:
            logger.info("Elitism selection...")
        sorted_population = sorted(self.population.chromosome, key=lambda x: x.fitness)
        return sorted_population[self.elitism_offset:self.elitism_offset + self.elitism_range]
    
    def tournament_selection(self) -> Chromosome:
        if settings.DEBUG:
            logger.info("Tournament selection...")
        selected_chromosome = []
        for _ in range(len(self.population.chromosome)):
            selected_chromosome.append(self.tournament_selection_helper())
        return selected_chromosome[0]
    
    def tournament_selection_helper(self) -> Chromosome:
        tournament_population = np.random.choice(self.population.chromosome, self.tournament_size)
        sorted_population = sorted(tournament_population, key=lambda x: x.fitness)
        return sorted_population[0]
    
    def roulette_selection(self) -> Chromosome:
        if settings.DEBUG:
            logger.info("Roulette selection...")
        fitness = [chromosome.fitness for chromosome in self.population.chromosome]
        total_fitness = sum(fitness)
        probabilities = [chromosome_fitness / total_fitness for chromosome_fitness in fitness]
        selected_chromosome = np.random.choice(self.population.chromosome, p=probabilities)
        return selected_chromosome
    
        
    


In [63]:
# #test Selection,
# factory = Factory()
# population = factory.generate_population(5)
# population.fitness_function = FitnessFunction
# population.calculate_fitness()

In [64]:
# selection = Selection(population, method="roulette")
# selection.select()

Test Selection

In [65]:
#test Selection
# factory = Factory()
# population = factory.generate_population(2)
# population.chromosome[0].genes
# population.chromosome[1].genes
# selection = Selection(population, method="roulette")
# a = selection.select()
    

### Builder

```python

In [66]:
from collections import defaultdict, namedtuple

class GeneticAlgorithmBuilder:
    '''
    Description:
    Genetic Algorithm Builder class for managing genetic algorithm parameters.
    '''
    def __init__(self):
        # Population parameters
        self.population_size = None
        self.max_generation = 10
        # Mutation parameters
        self.mutation_rate = 0.5
        self.mutation_size = 1
        # Crossover parameters
        self.crossover_probability = 0.5
        self.number_of_crossover_points = 1
        # Selection parameters
        self.tournament_size = 3
        self.elitism_size = 1
        self.elitism_offset = 0
        self.elitism_range = 1
        self.elitism_probability = 0.5
        self.selection_method = "tournament"
        # Dependency injection
        # Fitness function
        self.fitness_function = FitnessFunction()
        # Factory
        self.factory = Factory()
        # Population
        self.population = None
        # Mutation
        self.mutation = Mutation
        # Crossover
        self.crossover = Crossover
        # Selection
        self.selection = Selection
        # Repair
        self.repair = Repair
        # Logging
        self.logging = False
        # Data
        self._lab_data = LaboratoryData()
        self._module_data = ModuleData()
        self._group_data = GroupData()
        self._assistant_data = AssistantData()
        self._chapter_data = ChapterData()
    
    def set_mutations_class(self, mutation):
        self.mutation = mutation
        return self

    def set_mutations(self, mutation_rate, mutation_size):
        """
        Set mutation parameters
        :param mutation_rate: Mutation rate, probability of mutation
        :type mutation_rate: float
        :param mutation_size: Number of genes to mutate
        :type mutation_size: int
        :return: self
        """
        self.mutation_rate = mutation_rate
        self.mutation_size = mutation_size
        return self
    
    def get_mutations_parameters(self):
        return self.mutation_rate, self.mutation_size
    
    def set_crossover_class(self, crossover):
        self.crossover = crossover
        return self
    
    def set_crossover(self, crossover_probability, number_of_crossover_points):
        """
        Set crossover parameters
        :param crossover_probability: Probability of crossover
        :type crossover_probability: float
        :param number_of_crossover_points: Number of points to crossover
        :type number_of_crossover_points: int
        :return: self"""

        self.crossover_probability = crossover_probability
        self.number_of_crossover_points = number_of_crossover_points
        return self
    
    def get_crossover_parameters(self):
        return self.crossover_probability, self.number_of_crossover_points
    
    def set_selection_class(self, selection):
        self.selection = selection
        return self
    
    def set_selection(self, selection_method, **kwargs):

        """
        Set selection parameters
        :param selection_method: Selection method, "tournament", "elitism" or "roulette"
        :type selection_method: str
        :param kwargs: Selection parameters, tournament_size, elitism_size, elitism_offset, elitism_range, elitism_probability
        :type kwargs: dict
        :param tournament_size: Number of chromosomes to select for tournament selection
        :type tournament_size: int
        :param elitism_size: Number of chromosomes to select for elitism selection
        :type elitism_size: int
        :param elitism_offset: Offset for elitism selection, position of the first chromosome to select
        :type elitism_offset: int
        :param elitism_range: Range for elitism selection, number of chromosomes to select from the offset
        :type elitism_range: int
        :param elitism_probability: Probability of selecting from the elitism selection
        :type elitism_probability: float
        :return: self
        """
        

        self.selection_method = selection_method
        if selection_method == "tournament":
            self.tournament_size = kwargs.get("tournament_size", 3)
        elif selection_method == "elitism":
            self.elitism_size = kwargs.get("elitism_size", 1)
            self.elitism_offset = kwargs.get("elitism_offset", 0)
            self.elitism_range = kwargs.get("elitism_range", 1)
            self.elitism_probability = kwargs.get("elitism_probability", 0.5)
        elif selection_method == "roulette":
            pass
        else:
            raise ValueError("Invalid selection method.")
        return self
    
    def get_selection_parameters(self):
        return self.tournament_size, self.elitism_size, self.elitism_offset, self.elitism_range, self.elitism_probability
    
    def set_population_parameters(self, population_size, max_generation):
        """
        Set population parameters
        :param population_size: Population size
        :type population_size: int
        :param max_generation: Maximum number of generation
        :type max_generation: int
        :return: self
        """
        self.population_size = population_size
        self.max_generation = max_generation
        return self
    
    def get_population_parameters(self):
        return self.population_size, self.max_generation
    
    def set_data(self, lab_data, module_data, group_data, assistant_data, chapter_data):
        """
        Set data
        :param lab_data: Laboratory data
        :type lab_data: LaboratoryData
        :param module_data: Module data
        :type module_data: ModuleData
        :param group_data: Group data
        :type group_data: GroupData
        :param assistant_data: Assistant data
        :type assistant_data: AssistantData
        :param chapter_data: Chapter data
        :type chapter_data: ChapterData
        :return: self
        """
        self._lab_data = lab_data
        self._module_data = module_data
        self._group_data = group_data
        self._assistant_data = assistant_data
        self._chapter_data = chapter_data
        return self
    
    # def build(self):
    #     """
    #     Build genetic algorithm
    #     :return: GeneticAlgorithm
    #     """
    #     return GeneticAlgorithm(self)
    


In [67]:
#test GeneticAlgorithmBuilder
builder = GeneticAlgorithmBuilder()
builder

### Genetic Algorithm

```python

In [68]:
class GeneticAlgorithm:
    '''
    Description:
    Main class for genetic algorithm implementation
    '''
    def __init__(self, builder: GeneticAlgorithmBuilder):
        # Dependency injection
        self.builder = builder
        self.population_size, self.max_generation = self.builder.get_population_parameters()
        self.mutation_rate, self.mutation_size = self.builder.get_mutations_parameters()
        self.crossover_probability, self.number_of_crossover_points = self.builder.get_crossover_parameters()
        self.selection_method = self.builder.selection_method
        self.tournament_size, self.elitism_size, self.elitism_offset, self.elitism_range, self.elitism_probability = self.builder.get_selection_parameters()
        self.fitness_function = self.builder.fitness_function
        self.factory = self.builder.factory
        self.mutation = self.builder.mutation
        self.crossover = self.builder.crossover
        self.selection = self.builder.selection
        self.repair = self.builder.repair
        self.logging = self.builder.logging
        # Results
        self._results = defaultdict(list)
        self._best_chromosome = None
        self._best_fitness = None
        self._best_generation = None
        self._best_chromosome_history = []
        self._best_fitness_history = []

    def __repr__(self):
        return f"Population Size: {self.population_size}, Max Generation: {self.max_generation}, Mutation Rate: {self.mutation_rate}, Mutation Size: {self.mutation_size}, Crossover Probability: {self.crossover_probability}, Number of Crossover Points: {self.number_of_crossover_points}, Tournament Size: {self.tournament_size}, Elitism Size: {self.elitism_size}, Elitism Offset: {self.elitism_offset}, Elitism Range: {self.elitism_range}, Elitism Probability: {self.elitism_probability}, Selection Method: {self.selection_method}, Fitness Function: {self.fitness_function}, Factory: {self.factory}, Mutation: {self.mutation}, Crossover: {self.crossover}, Selection: {self.selection}, Repair: {self.repair}, Logging: {self.logging}, Results: {self._results}, Best Chromosome: {self._best_chromosome}, Best Fitness: {self._best_fitness}, Best Generation: {self._best_generation}, Best Chromosome History: {self._best_chromosome_history}, Best Fitness History: {self._best_fitness_history}"
    
    def __str__(self):
        return f"Population Size: {self.population_size}, Max Generation: {self.max_generation}, Mutation Rate: {self.mutation_rate}, Mutation Size: {self.mutation_size}, Crossover Probability: {self.crossover_probability}, Number of Crossover Points: {self.number_of_crossover_points}, Tournament Size: {self.tournament_size}, Elitism Size: {self.elitism_size}, Elitism Offset: {self.elitism_offset}, Elitism Range: {self.elitism_range}, Elitism Probability: {self.elitism_probability}, Selection Method: {self.selection_method}, Fitness Function: {self.fitness_function}, Factory: {self.factory}, Mutation: {self.mutation}, Crossover: {self.crossover}, Selection: {self.selection}, Repair: {self.repair}, Logging: {self.logging}, Results: {self._results}, Best Chromosome: {self._best_chromosome}, Best Fitness: {self._best_fitness}, Best Generation: {self._best_generation}, Best Chromosome History: {self._best_chromosome_history}, Best Fitness History: {self._best_fitness_history}"
    
    def initialize_population(self):
        if settings.DEBUG:
            logger.info("Initializing population...")
        population = self.factory.generate_population(self.population_size)
        population.fitness_function = self.fitness_function
        population.calculate_fitness()
        return population
    
    def loggings(self, population):
        if settings.DEBUG:
            logger.info("Logging...")
        self._results["best_chromosome"].append(population.chromosome[0])
        self._results["best_fitness"].append(population.chromosome[0].fitness)
        self._results["average_fitness"].append(sum([chromosome.fitness for chromosome in population.chromosome]) / len(population.chromosome))
        self._results["worst_fitness"].append(population.chromosome[-1].fitness)
        self._best_chromosome_history.append(population.chromosome[0])
        self._best_fitness_history.append(population.chromosome[0].fitness)
        return self
    
    def evolve(self, population):
        if settings.DEBUG:
            logger.info("Evolving...")
        new_population = Population([], None)

        # Choose the best chromosome from the previous generation
        new_population.chromosome.extend(self.selection(population, method='elitism', elitism_size=self.elitism_size, elitism_offset=self.elitism_offset, elitism_range=self.elitism_range, elitism_probability=self.elitism_probability).select())
        while len(new_population.chromosome) < self.population_size:
            parent1 = self.selection(population, method=self.selection_method, elitism_size=self.elitism_size, elitism_offset=self.elitism_offset, elitism_range=self.elitism_range, elitism_probability=self.elitism_probability).select()
            parent2 = self.selection(population, method=self.selection_method, elitism_size=self.elitism_size, elitism_offset=self.elitism_offset, elitism_range=self.elitism_range, elitism_probability=self.elitism_probability).select()
            child1, child2 = self.crossover(parent1, parent2, crossover_probability=self.crossover_probability, number_of_crossover_points=self.number_of_crossover_points).crossover()
            child1 = self.mutation(child1, mutation_rate=self.mutation_rate, mutation_size=self.mutation_size, factory=self.factory).mutate()
            child2 = self.mutation(child2, mutation_rate=self.mutation_rate, mutation_size=self.mutation_size, factory=self.factory).mutate()
            child2 = self.repair(child2, factory=self.factory).repair()
            child1 = self.repair(child1, factory=self.factory).repair()
            new_population.chromosome.extend([child1, child2])
        new_population.fitness_function = self.fitness_function
        new_population.calculate_fitness()
        
        if self.logging:
            self.loggings(new_population)

        return new_population
        

    def run(self):
        if settings.DEBUG:
            logger.info("Running...")
        population = self.initialize_population()
        if self.logging:
            self.loggings(population)
        for generation in range(self.max_generation):
            if settings.DEBUG:
                logger.info(f"Generation {generation}")
            population = self.evolve(population)
            if self.logging:
                self.loggings(population)
        self._best_chromosome = self._results["best_chromosome"][-1]
        self._best_fitness = self._results["best_fitness"][-1]
        self._best_generation = self._best_fitness_history.index(self._best_fitness)
        return self
    
    def get_results(self):
        return self._results
    
    def get_best_chromosome(self):
        return self._best_chromosome
    
    def get_best_fitness(self):
        return self._best_fitness
    
    def get_best_generation(self):
        return self._best_generation
    
    def get_best_chromosome_history(self):
        return self._best_chromosome_history
    
    def get_best_fitness_history(self):
        return self._best_fitness_history

Test Genetic Algorithm

In [69]:
#test GeneticAlgorithm
builder = GeneticAlgorithmBuilder()
builder.set_population_parameters(population_size=2, max_generation=2)
builder.set_mutations(mutation_rate=0.5, mutation_size=1)
builder.set_crossover(crossover_probability=0.5, number_of_crossover_points=1)
builder.set_selection(selection_method="tournament", tournament_size=3)
builder.set_data(lab_data=LaboratoryData(), module_data=ModuleData(), group_data=GroupData(), assistant_data=AssistantData(), chapter_data=ChapterData())
builder.fitness_function = FitnessFunction()
builder.fitness_function.max_groups_per_assistant = 100
builder.fitness_function.max_shift_per_assistant = 100
builder.factory = Factory()
builder.mutation = Mutation
builder.crossover = Crossover
builder.selection = Selection
builder.repair = Repair
builder.logging = True
algorithm = GeneticAlgorithm(builder)


In [70]:
# algorithm.run()

In [71]:
# algorithm.get_results()

# Tabu Search

```python

In [72]:
#tabu search
import collections
from copy import deepcopy
import numpy as np
import itertools
import random

In [73]:
# Tabu Search Operators
class TabuList:
    def __init__(self, tabu_list_size):
        self.tabu_list_size = tabu_list_size
        self.tabu_list = collections.deque(maxlen=tabu_list_size)
    
    def __repr__(self):
        return f"Tabu List Size: {self.tabu_list_size}, Tabu List: {self.tabu_list}"
    
    def __str__(self):
        return f"Tabu List Size: {self.tabu_list_size}, Tabu List: {self.tabu_list}"
    
    def add(self, item):
        self.tabu_list.append(item)
    
    def contains(self, item):
        return item in self.tabu_list
    
    def clear(self):
        self.tabu_list.clear()

In [74]:

class Neighborhood:
    def __init__(self, chromosome: Chromosome):
        self.chromosome = chromosome
        self.neighborhood = []
        self.method = None
        self.generate_neighborhood()
    
    def __repr__(self):
        return f"Chromosome: {self.chromosome}, Neighborhood: {self.neighborhood}"
    
    def __str__(self):
        return f"Chromosome: {self.chromosome}, Neighborhood: {self.neighborhood}"
    
    def swap_based_neighborhood(self):
        for i in range(len(self.chromosome.genes)):
            for j in range(i+1, len(self.chromosome.genes)):
                chromosome = deepcopy(self.chromosome)
                chromosome.genes[i], chromosome.genes[j] = chromosome.genes[j], chromosome.genes[i]
                self.neighborhood.append(chromosome)

    def insert_based_neighborhood(self):
        for i in range(len(self.chromosome.genes)):
            for j in range(len(self.chromosome.genes)):
                if i != j:
                    chromosome = deepcopy(self.chromosome)
                    chromosome.genes.insert(j, chromosome.genes.pop(i))
                    self.neighborhood.append(chromosome)

    def permutation_based_neighborhood(self):
        for i in range(len(self.chromosome.genes)):
            for j in range(len(self.chromosome.genes)):
                if i != j:
                    chromosome = deepcopy(self.chromosome)
                    chromosome.genes[i], chromosome.genes[j] = chromosome.genes[j], chromosome.genes[i]
                    self.neighborhood.append(chromosome)

    def perturbation_based_neighborhood(self):
        for i in range(len(self.chromosome.genes)):
            for j in range(len(self.chromosome.genes)):
                if i != j:
                    chromosome = deepcopy(self.chromosome)
                    chromosome.genes[i], chromosome.genes[j] = chromosome.genes[j], chromosome.genes[i]
                    self.neighborhood.append(chromosome)
        for i in range(len(self.chromosome.genes)):
            for j in range(i+1, len(self.chromosome.genes)):
                chromosome = deepcopy(self.chromosome)
                chromosome.genes[i], chromosome.genes[j] = chromosome.genes[j], chromosome.genes[i]
                self.neighborhood.append(chromosome)
        for i in range(len(self.chromosome.genes)):
            for j in range(len(self.chromosome.genes)):
                if i != j:
                    chromosome = deepcopy(self.chromosome)
                    chromosome.genes.insert(j, chromosome.genes.pop(i))
                    self.neighborhood.append(chromosome)

    def inverse_based_neighborhood(self):
        for i in range(len(self.chromosome.genes)):
            for j in range(i+1, len(self.chromosome.genes)):
                chromosome = deepcopy(self.chromosome)
                chromosome.genes[i:j+1] = chromosome.genes[i:j+1][::-1]
                self.neighborhood.append(chromosome)

    def vns_neighborhood(self):
        self.swap_based_neighborhood()
        self.insert_based_neighborhood()
        self.permutation_based_neighborhood()
        self.inverse_based_neighborhood()

    def generate_neighborhood(self):
        if self.method == "swap":
            self.swap_based_neighborhood()
        elif self.method == "insert":
            self.insert_based_neighborhood()
        elif self.method == "permutation":
            self.permutation_based_neighborhood()
        elif self.method == "perturbation":
            self.perturbation_based_neighborhood()
        elif self.method == "inverse":
            self.inverse_based_neighborhood()
        elif self.method == "vns":
            self.vns_neighborhood()
        else:
            raise ValueError("Invalid neighborhood method")

In [75]:
class SearchStrategy:
    def __init__(self, max_iteration):
        self.max_iteration = max_iteration
        self.iteration = 0
        self.best_solution = None
        self.best_solution_fitness = None

    def should_terminate(self):
        return self.iteration >= self.max_iteration

    def select_best_solution(self, neighborhood: Neighborhood, tabu_list: TabuList) -> Chromosome:
        best_solution = None
        best_solution_fitness = None
        for solution in neighborhood.neighborhood:
            if tabu_list.contains(solution):
                continue
            if best_solution is None or solution.fitness < best_solution_fitness:
                best_solution = solution
                best_solution_fitness = solution.fitness
        return best_solution

In [76]:
class FirstImprovementStrategy(SearchStrategy):
    def __init__(self, max_iteration):
        super().__init__(max_iteration)
    
    def search(self, neighborhood: Neighborhood, tabu_list: TabuList) -> Chromosome:
        while not self.should_terminate():
            best_solution = self.select_best_solution(neighborhood, tabu_list)
            if best_solution is None:
                break
            tabu_list.add(best_solution)
            neighborhood = Neighborhood(best_solution)
            self.iteration += 1
        return best_solution
    
    


In [77]:
class TabuSearchBuilder:
    def __init__(self):
        self.tabu_list_size = None
        self.max_iteration = None
        self.neighborhood_method = None
        self.search_strategy = None
        self.factory = Factory()
        self.fitness_function = FitnessFunction()
        self.tabu_list = None
        self.neighborhood = None
        self.logging = False
        self._results = defaultdict(list)
        self._best_chromosome = None
        self._best_fitness = None
        self._best_iteration = None
        self._best_chromosome_history = []
        self._best_fitness_history = []
    
    def set_tabu_list_size(self, tabu_list_size):
        self.tabu_list_size = tabu_list_size
        return self
    
    def set_max_iteration(self, max_iteration):
        self.max_iteration = max_iteration
        return self
    
    def set_neighborhood_method(self, neighborhood_method):
        self.neighborhood_method = neighborhood_method
        return self
    
    def set_search_strategy(self, search_strategy):
        self.search_strategy = search_strategy
        return self
    
    def set_factory(self, factory):
        self.factory = factory
        return self
    
    def set_fitness_function(self, fitness_function):
        self.fitness_function = fitness_function
        return self
    
    def set_logging(self, logging):
        self.logging = logging
        return self
    
    def get_results(self):
        return self._results
    
    def get_best_chromosome(self):
        return self._best_chromosome
    
    def get_best_fitness(self):
        return self._best_fitness
    
    def get_best_iteration(self):
        return self._best_iteration
    
    def get_best_chromosome_history(self):
        return self._best_chromosome_history
    
    def get_best_fitness_history(self):
        return self._best_fitness_history
    
    # def build(self):
    #     if self.tabu_list_size is None:
    #         raise ValueError("Tabu list size is not set")
    #     if self.max_iteration is None:
    #         raise ValueError("Max iteration is not set")
    #     if self.neighborhood_method is None:
    #         raise ValueError("Neighborhood method is not set")
    #     if self.search_strategy is None:
    #         raise ValueError("Search strategy is not set")
    #     if self.factory is None:
    #         raise ValueError("Factory is not set")
    #     if self.fitness_function is None:
    #         raise ValueError("Fitness function is not set")
    #     return TabuSearch(self)

In [78]:
class TabuSearch:
    def __init__(self, builder: TabuSearchBuilder):
        self.tabu_list_size = builder.tabu_list_size
        self.max_iteration = builder.max_iteration
        self.neighborhood_method = builder.neighborhood_method
        self.search_strategy = builder.search_strategy
        self.factory = builder.factory
        self.fitness_function = builder.fitness_function
        self.tabu_list = TabuList(self.tabu_list_size)
        self.neighborhood = None
        self.logging = builder.logging
        self._results = builder._results
        self._best_chromosome = builder._best_chromosome
        self._best_fitness = builder._best_fitness
        self._best_iteration = builder._best_iteration
        self._best_chromosome_history = builder._best_chromosome_history
        self._best_fitness_history = builder._best_fitness_history
    
    def __repr__(self):
        return f"Tabu List Size: {self.tabu_list_size}, Max Iteration: {self.max_iteration}, Neighborhood Method: {self.neighborhood_method}, Search Strategy: {self.search_strategy}, Factory: {self.factory}, Fitness Function: {self.fitness_function}, Tabu List: {self.tabu_list}, Neighborhood: {self.neighborhood}, Logging: {self.logging}, Results: {self._results}, Best Chromosome: {self._best_chromosome}, Best Fitness: {self._best_fitness}, Best Iteration: {self._best_iteration}, Best Chromosome History: {self._best_chromosome_history}, Best Fitness History: {self._best_fitness_history}"
    
    def __str__(self):
        return f"Tabu List Size: {self.tabu_list_size}, Max Iteration: {self.max_iteration}, Neighborhood Method: {self.neighborhood_method}, Search Strategy: {self.search_strategy}, Factory: {self.factory}, Fitness Function: {self.fitness_function}, Tabu List: {self.tabu_list}, Neighborhood: {self.neighborhood}, Logging: {self.logging}, Results: {self._results}, Best Chromosome: {self._best_chromosome}, Best Fitness: {self._best_fitness}, Best Iteration: {self._best_iteration}, Best Chromosome History: {self._best_chromosome_history}, Best Fitness History: {self._best_fitness_history}"
    
    def initialize(self, chromosome: Chromosome):
        self.neighborhood = Neighborhood(chromosome)
        self.search_strategy.best_solution = chromosome
        self.search_strategy.best_solution_fitness = chromosome.fitness
        self.tabu_list.clear()
        self.tabu_list.add(chromosome)
        return self
    
    def loggings(self, chromosome: Chromosome):
        self._results["best_chromosome"].append(chromosome)
        self._results["best_fitness"].append(chromosome.fitness)
        self._best_chromosome_history.append(chromosome)
        self._best_fitness_history.append(chromosome.fitness)
        return self
    
    def run(self, chromosome: Chromosome):
        self.initialize(chromosome)
        while not self.search_strategy.should_terminate():
            best_solution = self.search_strategy.search(self.neighborhood, self.tabu_list)
            if best_solution is None:
                break
            if best_solution.fitness < self.search_strategy.best_solution_fitness:
                self.search_strategy.best_solution = best_solution
                self.search_strategy.best_solution_fitness = best_solution.fitness
            self.tabu_list.add(best_solution)
            self.neighborhood = Neighborhood(best_solution)
            self.search_strategy.iteration += 1
            if self.logging:
                self.loggings(best_solution)
        self._best_chromosome = self._results["best_chromosome"][-1]
        self._best_fitness = self._results["best_fitness"][-1]
        self._best_iteration = self._best_fitness_history.index(self._best_fitness)
        return self
    
    def get_results(self):
        return self._results
    
    def get_best_chromosome(self):
        return self._best_chromosome
    
    def get_best_fitness(self):
        return self._best_fitness